In [1]:
%cd /home/baosiek/Projects/tchumyt-yofy/

/home/baosiek/Projects/tchumyt-yofy


In [2]:
from llm.llm.architecture.gpt_model import GPTModel
from llm.llm import cfg
from llm.llm.pipelines.inferencing.text_inferencer import TextProvider
import torch

torch.manual_seed(123)


2025-01-21 09:34:57 - __init__ - INFO - Configuration file: llm/configs/gpt_config_124m_test.yaml


In [3]:
input_sentences = ["every effort moves", "I really like"]
target_sentences = [" effort moves you", " really like chocolate"]

In [4]:
model = GPTModel(cfg=cfg)

In [5]:
text_provider = TextProvider(cfg=cfg)

In [6]:
inputs = torch.tensor([], dtype=int)
targets = torch.tensor([], dtype=int)

for sent in input_sentences:
    tk_ids = text_provider.text_to_token_ids(sent)
    inputs = torch.cat((inputs, tk_ids))

for sent in target_sentences:
    tk_ids = text_provider.text_to_token_ids(sent)
    targets = torch.cat((targets, tk_ids))

print(f"Inputs: \n{inputs}")
print(f"Targets: \n{targets}")

Inputs: 
tensor([[16833,  3626,  6100],
        [   40,  1107,   588]])
Targets: 
tensor([[ 3626,  6100,   345],
        [ 1107,   588, 11311]])


In [39]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1)
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print(f"token ids: \n{token_ids}")

token ids: 
tensor([[[36595],
         [18824],
         [28544]],

        [[35098],
         [21445],
         [ 8533]]])


In [40]:
print(f"Targets sentence 1: {text_provider.token_ids_to_text(targets[0])}")
print(f"Output model sentence 1: {text_provider.token_ids_to_text(token_ids=token_ids[0].flatten())}")
print(f"Targets sentence 2: {text_provider.token_ids_to_text(targets[1])}")
print(f"Output model sentence 2: {text_provider.token_ids_to_text(token_ids=token_ids[1].flatten())}")

Targets sentence 1:  effort moves you
Output model sentence 1:  Rookie crapIncreases
Targets sentence 2:  really like chocolate
Output model sentence 2: ㅋ tenants Dallas


In [82]:
text_idx = 0
target_probas_1 = probas[text_idx, [0,1,2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0,1,2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([9.6694e-06, 2.1944e-05, 2.4710e-05])
Text 2: tensor([2.5327e-05, 8.8794e-06, 2.1665e-05])


In [85]:
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(f"Log probabilities: {log_probas}")

avg_log = torch.mean(log_probas)
print(f"Average Log probabilities: {avg_log}")

neg_avg_log = avg_log * -1
print(f"Negative Average Log probabilities: {neg_avg_log}")

Log probabilities: tensor([-11.5465, -10.7270, -10.6083, -10.5836, -11.6318, -10.7398])
Average Log probabilities: -10.97284984588623
Negative Average Log probabilities: 10.97284984588623


In [87]:
print(f"Shape target tensors: {targets.shape}")
print(f"Shape logits tensors: {logits.shape}")

Shape target tensors: torch.Size([2, 3])
Shape logits tensors: torch.Size([2, 3, 50257])


In [90]:
targets_flat = targets.flatten()
logits_flat = logits.flatten(0, 1)

print(f"Shape flatten target tensors: {targets_flat.shape}")
print(f"Shape flatten logits tensors: {logits_flat.shape}")

Shape flatten target tensors: torch.Size([6])
Shape flatten logits tensors: torch.Size([6, 50257])


In [91]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(f"Loss: {loss}")

Loss: 10.972847938537598
